# Практическая работа 13

In [61]:
import pandas as pd
import tqdm
import numpy as np
import time
import plotly.express as px
from apriori_python import apriori as apriori_1
from apyori import apriori as apriori_2
from efficient_apriori import apriori as apriori_3
from fpgrowth_py import fpgrowth

1. Загрузить данные Market_Basket_Optimisation.csv.
2. Визуализировать данные (отразить на гистограммах относительную и фактическую частоту встречаемости для 20 наиболее популярных товаров).

In [63]:
def plot_all(filename: str):
    file1 = open(filename, "r")
    lines = file1.readlines()

    df_market = dict()

    for line in tqdm.tqdm(lines):
        line = line.replace("\n", "")
        products = line.split(",")
        for product in products:
            product = product.replace(" ", "")
            if product not in df_market.keys():
                df_market[product] = 1
            else:
                df_market[product] += 1

    df = {"product": list(), "count": list()}
    for key in df_market.keys():
        df["product"].append(key)
        df["count"].append(df_market[key])
    df_market = pd.DataFrame.from_dict(df)
    df_market = df_market.sort_values(by="count", ascending=False).reset_index(drop=True)

    df_market['std'] = np.nan
    total = df_market['count'].sum()
    for i in range(len(df_market['product'])):
        df_market['std'][i] = df_market['count'][i]/total

    market_top_20 = df_market.head(20)
    fig = px.bar(market_top_20, x='product', y='count', title="Общее распределение количества")
    fig.show()
    fig = px.bar(market_top_20, x='product', y='std', title="Относительная частота")
    fig.show()

    buffer = pd.DataFrame(market_top_20, columns=['count'])
    buffer.rename(columns={'count': 'fact'})
    market_top_20['fact'] = buffer.apply(lambda item: item / (buffer.shape[0]*100))
    fig = px.bar(market_top_20, x='product', y='fact', title="Фактическая частота")
    fig.show()

plot_all(filename="data/13_practice/Market_Basket_Optimisation.csv")

100%|██████████| 7501/7501 [00:00<00:00, 535751.55it/s]
C:\Users\orono\AppData\Local\Temp\ipykernel_2164\2272681559.py:27: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



C:\Users\orono\AppData\Local\Temp\ipykernel_2164\2272681559.py:37: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [64]:
df = pd.read_csv("data/13_practice/Market_Basket_Optimisation.csv", header=None)
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
1,burgers,meatballs,eggs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,chutney,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,turkey,avocado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mineral water,milk,energy bar,whole wheat rice,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7496,butter,light mayo,fresh bread,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7497,burgers,frozen vegetables,eggs,french fries,magazines,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7498,chicken,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7499,escalope,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


В большинстве библиотек, которые реализуют алгоритмы поиска ассоциативных правил в качестве входных значений необходимо подавать список транзакций, то есть список списков, поэтому нам необходимо преобразовать наш датасет в такой формат. При этом необходимо контролировать пустые значения в данных (NaN), их добавлять в список не надо.

3. Применить алгоритм Apriori, используя 3 разные библиотеки (apriori_python, apyori, efficient_apriori).
4. Применить алгоритм FP-Growth из библиотеки fpgrowth_py.
5. Сравнить время выполнения всех алгоритмов и построить гистограмму.

In [16]:
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
1,burgers,meatballs,eggs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,chutney,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,turkey,avocado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mineral water,milk,energy bar,whole wheat rice,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7496,butter,light mayo,fresh bread,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7497,burgers,frozen vegetables,eggs,french fries,magazines,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7498,chicken,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7499,escalope,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**Настраиваемые параметры**

**Support** - показатель «частотности» данного itemset во всех анализируемых транзакциях
**Confidence** - Это показатель того, как часто наше правило срабатывает для всего датасета.
**Lift** - это отношение «зависимости» items к их «независимости». Lift показывает, насколько items зависят друг от друга.
**Conviction** (убедительность) — это «частотность ошибок» нашего правила.

In [54]:
def show_results(result: list, num: float):
    print(f"Method {num} results:")
    for rule in result:
        print(f"\t>> {rule}")


class Apriori:
    def __init__(self, df: pd.DataFrame):
        self.transactions = []
        for i in range(df.shape[0]):
            row = df.iloc[i].dropna().tolist()
            self.transactions.append(row)
        self.time_ranges = dict()

    def __call__(self, minsup: float, minconf: float, *args, **kwargs):
        """Return all apriori-things"""
        self.minsup = minsup
        self.minconf = minconf
        self._method_1()
        self._method_2()
        self._method_3()
        self._fpgrowth()

        df_times = pd.DataFrame({'name': self.time_ranges.keys(), 'time': self.time_ranges.values()})
        fig = px.bar(df_times, x='name', y='time', title="Время алгоритмов")
        fig.show()

    def _method_1(self):
        start=time.perf_counter()
        t1, rules = apriori_1(self.transactions, minSup=self.minsup, minConf=self.minconf)
        self.time_ranges['apriori_python'] = time.perf_counter()-start
        show_results(rules, 1)

    def _method_2(self):
        start=time.perf_counter()
        rules = apriori_2(self.transactions, min_support=self.minsup,
                          min_confidence=self.minconf, min_lift=1.0001)
        results = list(rules)
        self.time_ranges['apyori'] = time.perf_counter()-start
        show_results(results, 2)

    def _method_3(self):
        start=time.perf_counter()
        itemsets, rules = apriori_3(self.transactions, min_support=self.minsup, min_confidence=self.minconf)
        self.time_ranges['efficient_apriori'] = time.perf_counter()-start
        show_results(rules, 3)

    def _fpgrowth(self):
        start=time.perf_counter()
        itemsets, rules = fpgrowth(self.transactions, minSupRatio=self.minsup, minConf=self.minconf)
        self.time_ranges['fpgrowth'] = time.perf_counter()-start
        show_results(rules, 4)

time_ranges = Apriori(df=df)(minsup=0.02, minconf=0.4)

Method 1 results:
	>> [{'ground beef'}, {'mineral water'}, 0.41655359565807326]
	>> [{'olive oil'}, {'mineral water'}, 0.4190283400809717]
	>> [{'soup'}, {'mineral water'}, 0.45646437994722955]
Method 2 results:
	>> RelationRecord(items=frozenset({'mineral water', 'ground beef'}), support=0.040927876283162246, ordered_statistics=[OrderedStatistic(items_base=frozenset({'ground beef'}), items_add=frozenset({'mineral water'}), confidence=0.41655359565807326, lift=1.7475215442008991)])
	>> RelationRecord(items=frozenset({'mineral water', 'olive oil'}), support=0.027596320490601255, ordered_statistics=[OrderedStatistic(items_base=frozenset({'olive oil'}), items_add=frozenset({'mineral water'}), confidence=0.4190283400809717, lift=1.7579035676439423)])
	>> RelationRecord(items=frozenset({'mineral water', 'soup'}), support=0.023063591521130515, ordered_statistics=[OrderedStatistic(items_base=frozenset({'soup'}), items_add=frozenset({'mineral water'}), confidence=0.45646437994722955, lift=1.91

6. Загрузить данные data.csv.

7. Визуализировать данные (отразить на гистограммах относительную и фактическую частоту встречаемости для 20 наиболее популярных товаров).

In [65]:
plot_all(filename="data/13_practice/data.csv")

100%|██████████| 9531/9531 [00:00<00:00, 476550.45it/s]
C:\Users\orono\AppData\Local\Temp\ipykernel_2164\2272681559.py:27: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



C:\Users\orono\AppData\Local\Temp\ipykernel_2164\2272681559.py:37: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



8. Применить алгоритм Apriori, используя 3 разные библиотеки (apriori_python, apyori, efficient_apriori).
9. Применить алгоритм FP-Growth из библиотеки fpgrowth_py.
10. Сравнить время выполнения всех алгоритмов и построить гистограмму.

In [60]:
df = pd.read_csv("data/13_practice/data.csv", header=None)
time_ranges = Apriori(df=df)(minsup=0.02, minconf=0.4)

Method 1 results:
	>> [{'Hot chocolate'}, {'Coffee'}, 0.5072463768115942]
	>> [{'Cookies'}, {'Coffee'}, 0.5184466019417475]
	>> [{'Cake'}, {'Coffee'}, 0.5269582909460834]
	>> [{'Sandwich'}, {'Coffee'}, 0.5323529411764706]
	>> [{'Juice'}, {'Coffee'}, 0.5342465753424658]
	>> [{'Pastry'}, {'Coffee'}, 0.5521472392638037]
	>> [{'Medialuna'}, {'Coffee'}, 0.5692307692307692]
	>> [{'Toast'}, {'Coffee'}, 0.7044025157232704]
Method 2 results:
	>> RelationRecord(items=frozenset({'Coffee', 'Cake'}), support=0.054348966530269646, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Cake'}), items_add=frozenset({'Coffee'}), confidence=0.5269582909460834, lift=1.1091959962471556)])
	>> RelationRecord(items=frozenset({'Coffee', 'Cookies'}), support=0.028013849543594586, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Cookies'}), items_add=frozenset({'Coffee'}), confidence=0.5184466019417476, lift=1.0912797180006175)])
	>> RelationRecord(items=frozenset({'Coffee', 'Hot chocolate'}), 